In [28]:
from tkinter import *
from collections import deque
import random
import sys

WIDTH = 900
HEIGHT = 600
POLE_WIDTH = 30
POLE_HEIGHT = 400
POLE1_BASE = (WIDTH/4,HEIGHT/2+POLE_HEIGHT/2)
POLE2_BASE = (WIDTH/2,HEIGHT/2+POLE_HEIGHT/2)
POLE3_BASE = (WIDTH*(3/4),HEIGHT/2+POLE_HEIGHT/2)
POLE_BASES = [POLE1_BASE,POLE2_BASE,POLE3_BASE]
TILE_WIDTH = 50
TILE_HEIGHT = 20
TILE_SCALING = 5
BUFFER_POS = [WIDTH/2,(1/9)* HEIGHT]
TILE_COLORS = ['#ffff00','#ff7700']
SELECTION_MAX = 100
ENTRY_TEXT = "Enter # of Tiles (1-20)"
DELAY = 200
TARGET_TEXT_PADDING = 10
DUMMY_EVENT = Event()
DUMMY_EVENT.char = ""

class Hanoi:
    def __init__(self):
        self.window = Tk()
        self.window.title('Hanoi')
        self.canvas = Canvas(self.window, width=WIDTH, height=HEIGHT, bg="#36f5ce")
        self.canvas.bind("<Button-1>",self.handleClick)
        self.canvas.bind("<Motion>",self.handleMotion)
        self.canvas.bind("<B1-Motion>",self.handleMotion)
        self.canvas.bind("<ButtonRelease-1>",self.handleDragRelease)
        self.canvas.pack()
        self.L1 = Label(self.window, font="Courier")
        self.L1.pack()
        self.E1 = Entry(self.window, bg="#36f5ce", font="Courier", width=len(ENTRY_TEXT))
        self.E1.bind('<Key>',self.draw)
        self.E1.insert(0, ENTRY_TEXT)
        self.E1.selection_range(0,SELECTION_MAX)
        self.E1.focus_set()
        self.E1.pack(side=LEFT)
        self.B1 = Button(self.window,command=self.solve,activebackground="#b8b8b8",bg="#4287f5",text="Solve")
        self.B1.pack(side=LEFT)
        self.B2 = Button(self.window,command=self.window.destroy,text="Quit")
        self.B2.pack(side=RIGHT)
        self.B3 = Button(self.window,command=self.fillRandom,activebackground="#b8b8b8",bg="#4287f5",text="Arrange Randomly")
        self.B3.pack(side=LEFT)
        self.S1 = Scale(self.window,from_=1,to=1000,orient=HORIZONTAL,length=400, command=self.setDelay,label='Delay per Move (ms)',
            resolution=1, bg="#36f5ce", font="Courier")
        self.S1.set(200)
        self.S1.pack(side=BOTTOM)
        
        self.delay = DELAY
        self.n = 0
        self.towers = None
        self.tiles = None
        self.ops = 0
        self.origin = 0
        self.target = 2
        self.stopped_flag = True
        self.tileIsGrabbed = False
        self.grabbedTile = None
        self.hoveredTower = None
        self.previousTower = None
        self.dragging = False
        self.mode = "classic"

        self.draw(event=DUMMY_EVENT)

    def solve(self):
        if not self.stopped_flag: return
        self.tileIsGrabbed = False
        self.grabbedTile = None
        self.ops = 0
        if len(self.towers[0]) == self.n: self.mode = "classic"
        else: self.mode = "random"

        ai = HanoiAI(self)
        if self.mode == "classic":
            self.draw(event=DUMMY_EVENT)
            moves = ai.AIsolveClassic()
        elif self.mode == "random":
            moves = ai.AIsolveRandom()
        self.stopped_flag = False
        self.playSolution(moves, len(moves), 0)
        
    def playSolution(self, moves, n, i):
        if i < n and not self.stopped_flag:
            self.move(moves[i][0],moves[i][1])
            self.window.after(self.delay,self.playSolution,moves,n,i+1)
        else:
            self.stopped_flag = True

    def draw(self, event=None):
        self.canvas.delete('all')
        self.drawPoles()
        self.drawTiles(event)


    def drawPoles(self):
        self.canvas.create_rectangle(WIDTH/4-POLE_WIDTH/2,HEIGHT/2-POLE_HEIGHT/2,WIDTH/4+POLE_WIDTH/2,HEIGHT/2+POLE_HEIGHT/2,fill="#5c2715",outline="black",width=2)
        self.canvas.create_rectangle(WIDTH/2-POLE_WIDTH/2,HEIGHT/2-POLE_HEIGHT/2,WIDTH/2+POLE_WIDTH/2,HEIGHT/2+POLE_HEIGHT/2,fill="#5c2715",outline="black",width=2)
        self.canvas.create_rectangle(WIDTH*(3/4)-POLE_WIDTH/2,HEIGHT/2-POLE_HEIGHT/2,WIDTH*(3/4)+POLE_WIDTH/2,HEIGHT/2+POLE_HEIGHT/2,fill="green",outline="black",width=2)
        self.canvas.create_text(POLE3_BASE[0], POLE3_BASE[1] + TARGET_TEXT_PADDING, text="TARGET", tags="target_text", font="(Courier,30,bold)")

    def drawTiles(self, event=None):
        if event: 
            self.stopped_flag = True
            self.n = self.handleEntry(event)
            self.resetTowers()
        self.tiles = [None]*self.n
        ops_string = "moves: " + str(self.ops)
        self.canvas.create_text(WIDTH-60,30,text=ops_string, tags="ops_text",font="(Courier,30,bold)")
        for t, tower in enumerate(self.towers):
            t_height = len(tower)
            for i in range(t_height):
                t_ = tower[i] - 1
                self.tiles[t_] = [POLE_BASES[t][0], POLE_BASES[t][1]-TILE_HEIGHT/2-(t_height-i-1)*TILE_HEIGHT]
                self.canvas.create_rectangle(self.tiles[t_][0]-(t_*TILE_SCALING+TILE_WIDTH/2), self.tiles[t_][1]-TILE_HEIGHT/2, 
                    self.tiles[t_][0]+(t_*TILE_SCALING+TILE_WIDTH/2), self.tiles[t_][1]+TILE_HEIGHT/2,
                    outline="#000",fill=TILE_COLORS[tower[i]%2], width=2)
                self.canvas.create_text(self.tiles[t_][0], self.tiles[t_][1], text=str(tower[i]), tags="tile_label", font="(Courier,30,bold)")

    def drawBufferTile(self):
        self.canvas.create_rectangle(POLE_BASES[self.hoveredTower][0]-(self.grabbedTile*TILE_SCALING+TILE_WIDTH/2),POLE_BASES[self.hoveredTower][1]-(POLE_HEIGHT+POLE_WIDTH)-TILE_HEIGHT/2,
            POLE_BASES[self.hoveredTower][0]+(self.grabbedTile*TILE_SCALING+TILE_WIDTH/2),POLE_BASES[self.hoveredTower][1]-(POLE_HEIGHT+POLE_WIDTH)+TILE_HEIGHT/2,outline="#000",fill=TILE_COLORS[self.grabbedTile%2], width=2)
        self.canvas.create_text(POLE_BASES[self.hoveredTower][0], POLE_BASES[self.hoveredTower][1]-(POLE_HEIGHT+POLE_WIDTH), text=str(self.grabbedTile), tags="target_text", font="(Courier,30,bold)")


    def handleEntry(self,event=None):
        self.mode = "classic"
        input = ""
        if event:
            if event.char == "\r":
                self.E1.selection_range(0,SELECTION_MAX)
                return int(self.E1.get())
            elif event.char == "\x7f":
                self.E1.delete(0,SELECTION_MAX)
                return 0
            elif event.char in [str(i) for i in range(10)]:
                input = event.char
        prefix = self.E1.get()
        if prefix == ENTRY_TEXT:
            prefix = ""
        if prefix + input not in [str(i) for i in range(1,21)]:
            return 0
        return int(prefix + input)

    def handleClick(self, e):
        tower = int(e.x // (WIDTH/3))
        if self.n < 1 or not self.stopped_flag: return
        if not self.tileIsGrabbed:
            if not self.towers[tower]: return
            self.grabbedTile = self.towers[tower].popleft()
            self.tileIsGrabbed = True
            self.hoveredTower = tower
            self.previousTower = tower
            self.draw()
            self.drawBufferTile()
        else:
            if self.towers[tower] and self.towers[tower][0] < self.grabbedTile: return
            self.towers[tower].appendleft(self.grabbedTile)
            self.grabbedTile = None
            self.tileIsGrabbed = False
            self.ops += tower != self.previousTower
            self.draw()

    def handleMotion(self, e):
        if not self.tileIsGrabbed: return
        self.dragging = e.state == 256
        tower = int(e.x // (WIDTH/3))
        if tower != self.hoveredTower:
            self.hoveredTower = tower
            self.draw()
            self.drawBufferTile()

    def handleDragRelease(self,e):
        if self.n < 1 or not self.stopped_flag: return
        tower = self.hoveredTower
        if self.dragging:
            if self.towers[tower] and self.towers[tower][0] < self.grabbedTile: 
                self.towers[self.previousTower].appendleft(self.grabbedTile)
                self.draw()
            else:
                self.towers[tower].appendleft(self.grabbedTile)
                self.ops += tower != self.previousTower
                self.draw()
            self.grabbedTile = None
            self.dragging = self.tileIsGrabbed = False

    def setDelay(self, delay):
        self.delay = delay

    def resetTowers(self):
        self.ops = 0
        self.towers = [deque([i for i in range(1,self.n+1)]), deque(), deque()]
    
    def fillRandom(self, event=None):
        self.tileIsGrabbed = False
        self.grabbedTile = None
        self.mode = "random"
        self.towers = [deque(),deque(),deque()]
        for i in range(1,self.n+1):
            self.towers[random.randint(0,2)].append(i)
        self.ops = 0
        self.draw()

    def move(self,origin,target):
        self.ops += 1
        self.towers[target].appendleft(self.towers[origin].popleft())
        self.draw()

    def mainloop(self):
        self.window.mainloop()



class HanoiAI:
    def __init__(self, context: 'Hanoi'):
        self.ctx = context
        self.t_heights = None
        self.t_idxs = None
        self.moves = []

    def AIsolveRandom(self):
        self.t_heights = [len(t) for t in self.ctx.towers]
        self.t_idxs = [h-1 for h in self.t_heights]
        self.recRandom(self.ctx.n, self.ctx.target)
        return self.moves

    def recRandom(self, n, target):
        if n == 0: return
        origin = self.findTile(n)
        if origin == target:
            self.recRandom(n-1,target)
        else:
            helper = 3 - origin - target
            self.recRandom(n-1, helper)
            self.moves.append([origin,target])
            self.recClassic(n-1,helper,target)

    def findTile(self, n):
        for i, tower in enumerate(self.ctx.towers):
            if self.t_idxs[i] >= 0:
                if tower[self.t_idxs[i]] == n:
                    self.t_idxs[i] -= 1
                    return i

        
    def AIsolveClassic(self):
        self.recClassic(self.ctx.n, self.ctx.origin, self.ctx.target)
        return self.moves

    def recClassic(self, n, origin, target):
        if n == 0: return
        helper = 3 - origin - target
        self.recClassic(n-1, origin, helper)
        self.moves.append([origin,target])
        self.recClassic(n-1,helper,target)
        
        


hanoi = Hanoi()
hanoi.mainloop()


